<a id='top'></a>
# Db2 Audit Facility

To manage access to your sensitive data, you can use a variety of authentication and access control mechanisms to establish rules and controls for acceptable data access. But to protect against and discover unknown or unacceptable behaviors you can monitor data access by using the Db2® audit facility.

<a id='top'></a>
## Table of Contents

* [Introduction](#intro)
* [Audit tables](#tables)
* [Categories](#categories)
* [Define Auditing Rules](#define)
* [Categories](#categories)
* [Effect of switching user](#user_switch)
<p>
* [Catalog Views](#catalog)
* [Answers](#answers)
* [Clean all](#clean)

In [ ]:
%run db2.ipynb

## Introduction <a id='intro'></a>

The audit facility provides the ability to audit at both the instance and the individual database level, independently recording all instance and database level activities with separate logs for each. The system administrator (in our scenario: `Peter` who holds `SYSADM` authority) can use the db2audit tool to configure audit at the instance level as well as to control when such audit information is collected. The system administrator can use the db2audit tool to archive both instance and database audit logs as well as to extract audit data from archived logs of either type.

The security administrator (in our scenario: `Alex` who holds `SECADM` authority within a database) can use audit policies in conjunction with the SQL statement, AUDIT, to configure and control the audit requirements for an individual database. The security administrator can use the following audit routines to perform the specified tasks:
The SYSPROC.AUDIT_ARCHIVE stored procedure archives audit logs.
The SYSPROC.AUDIT_LIST_LOGS table function allows you to locate logs of interest.
The SYSPROC.AUDIT_DELIM_EXTRACT stored procedure extracts data into delimited files for analysis.
The security administrator can grant EXECUTE privilege on these routines to another user, therefore enabling the security administrator to delegate these tasks, if required.

## Audit tables <a id='tables'></a>
Before you can work with audit data in database tables, you need to create the tables to hold the data. We are creating these tables in a separate schem (here `audit `) to isolate the data in the tables from unauthorized users.

After you have created the tables, the security administrator can use the SYSPROC.AUDIT_DELIM_EXTRACT stored procedure, or the system administrator can use the db2audit extract command, to extract the audit records from the archived audit log files into delimited files. You can load the audit data from the delimited files into the database tables you just created.

In [3]:
%run ../db2.ipynb

DB2 Extensions Loaded.


In [14]:
%sql CONNECT TO SAMPLE user peter using p123

Connection successful.


In [29]:
%sql SET CURRENT SCHEMA audit

Command completed.


In [32]:
%run db2audit.ipynb

Command completed.


In [33]:
%%sql
SELECT tabschema,
       tabname
    FROM syscat.tables
    WHERE tabschema = 'AUDIT'

,TABSCHEMA,TABNAME
0,AUDIT,AUDIT
1,AUDIT,CHECKING
2,AUDIT,CONTEXT
3,AUDIT,EXECUTE
4,AUDIT,OBJMAINT
5,AUDIT,SECMAINT
6,AUDIT,SYSADMIN
7,AUDIT,VALIDATE


In [34]:
%sql CONNECT RESET

Connection closed.


[Back to Top](#top)

## Categories of audit records <a id='categories'></a>

There are different categories of audit records that may be generated. In the following description of the categories of events available for auditing, you should notice that following the name of each category is a one-word keyword used to identify the category type. 

#### AUDIT
Generates records when audit settings are changed or when the audit log is accessed. Following is an example how to define an audit policy.

In [102]:
%sql CONNECT TO SAMPLE user alex using p123

Connection successful.


In [48]:
%%sql
CREATE AUDIT POLICY Policy_Audit
     CATEGORIES AUDIT STATUS BOTH
     ERROR TYPE NORMAL;

Command completed.


In [49]:
%sql DROP AUDIT POLICY Policy_Audit

Command completed.


#### Authorization Checking (CHECKING)
Generates records during authorization checking of attempts to access or manipulate Db2 database objects or functions.

#### Object Maintenance (OBJMAINT)
Generates records when creating or dropping data objects, and when altering certain objects.

#### Security Maintenance (SECMAINT). Generates records when:
Granting or revoking object privileges or database authorities
Granting or revoking security labels or exemptions
Altering the group authorization, role authorization, or override or restrict attributes of an LBAC security policy
Granting or revoking the SETSESSIONUSER privilege
Modifying any of the SYSADM_GROUP, SYSCTRL_GROUP, SYSMAINT_GROUP, or SYSMON_GROUP configuration parameters.
#### System Administration (SYSADMIN)
Generates records when operations requiring SYSADM, SYSMAINT, or SYSCTRL authority are performed.
#### User Validation (VALIDATE)
Generates records when authenticating users or retrieving system security information.
#### Operation Context (CONTEXT)
Generates records to show the operation context when a database operation is performed. This category allows for better interpretation of the audit log file. When used with the log's event correlator field, a group of events can be associated back to a single database operation. For example, a query statement for dynamic queries, a package identifier for static queries, or an indicator of the type of operation being performed, such as CONNECT, can provide needed context when analyzing audit results.
#### Execute (EXECUTE)
Generates records during the execution of SQL statements.

## Define Auditing Rules <a id='define'></a>

#### Auditing any actions by SYSADM or DBADM
In order to complete their security compliance certification, a company must show that any and all activities within the database by those people holding system administration (SYSADM) or database administrative (DBADM) authority can be monitored.

To capture all actions within the database, both the EXECUTE and SYSADMIN categories should be audited. The security administrator creates an audit policy that audits these two categories. The security administrator can use the AUDIT statement to associate this audit policy with the SYSADM and DBADM authorities. Any user that holds either SYSADM or DBADM authority will then have any auditable events logged. The following example shows how to create such an audit policy and associate it with the SYSADM and DBADM authorities:

In [51]:
%%sql
CREATE AUDIT POLICY AdminsPolicy 
    CATEGORIES EXECUTE STATUS BOTH,
    SYSADMIN STATUS BOTH 
    ERROR TYPE AUDIT


Command completed.


Activate audit policy

In [58]:
%sql AUDIT SYSADM, DBADM USING POLICY ADMINSPOLICY

Command completed.


Check the policies in the Db2 catalog tables.

In [62]:
%sql -grid SELECT * FROM SYSCAT.AUDITPOLICIES

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [63]:
%sql -grid SELECT * FROM SYSCAT.AUDITUSE

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

#### Auditing any access to a specific table
Consider a company where the EMPLOYEE table contains extremely sensitive information and the company wants to audit any and all SQL access to the data in that table. The EXECUTE category can be used to track all access to a table; it audits the SQL statement, and optionally the input data value provided at execution time for that statement.

There are two steps to track activity on the table. First, the security administrator creates an audit policy that specifies the EXECUTE category, and then the security administrator associates that policy with the table:


In [60]:
%%sql
CREATE AUDIT POLICY SensitiveDataPolicy
    CATEGORIES EXECUTE 
    STATUS BOTH 
    ERROR TYPE AUDIT

Command completed.


In [61]:
%sql AUDIT TABLE HMO.PATIENT USING POLICY SensitiveDataPolicy


Command completed.


#### SECMAINT
...


In [ ]:
%%sql
CREATE AUDIT POLICY SensitiveDataPolicy
    CATEGORIES SECMAINT 
    STATUS BOTH 
    ERROR TYPE AUDIT

#### auditing for a database

In [103]:
%%sql
CREATE AUDIT POLICY ALTPOLICY 
  CATEGORIES 
    AUDIT STATUS BOTH, 
    OBJMAINT STATUS BOTH,  
    CHECKING STATUS BOTH, 
    EXECUTE STATUS BOTH 
  ERROR TYPE NORMAL;



Command completed.


In [105]:
%sql AUDIT DATABASE USING POLICY ALTPOLICY

Command completed.


## Inspect audit logs

In [ ]:
db2audit flush

db2audit archive database sample to ./auditarchive

db2audit extract delasc to ./audit_del_files/ from files db2audit.db.SAMPLE.log.0.20190809104637135267.bk 



CALL SYSPROC.AUDIT_ARCHIVE( '/auditarchive', -2 )

CALL SYSPROC.AUDIT_DELIM_EXTRACT(
     '', '', '/auditarchive',  'db2audit.%.200604%', '' ) 

CALL SYSPROC.AUDIT_DELIM_EXTRACT( '', '', '/auditarchive', 
    'db2audit.%.20060419034937', 'category 
     execute status success, checking status failure );

LOAD FROM audit.del OF DEL MODIFIED BY DELPRIORITYCHAR LOBSINFILE
             INSERT INTO audit.AUDIT


In [107]:
%sql CONNECT TO SAMPLE user peter using p123

Connection successful.


In [114]:
%sql CALL ADMIN_CMD('LOAD FROM /home/db2inst1/sqllib/security/auditdata/audit_del_files/audit.del OF DEL MODIFIED BY DELPRIORITYCHAR  LOBSINFILE INSERT INTO audit.AUDIT');

,ROWS_READ,ROWS_SKIPPED,ROWS_LOADED,ROWS_REJECTED,ROWS_DELETED,ROWS_COMMITTED,ROWS_PARTITIONED,NUM_AGENTINFO_ENTRIES,MSG_RETRIEVAL,MSG_REMOVAL
0,0,0,0,0,0,0,None,None,None,None


In [115]:
%sql CALL ADMIN_CMD('LOAD FROM /home/db2inst1/sqllib/security/auditdata/audit_del_files/checking.del OF DEL MODIFIED BY DELPRIORITYCHAR  LOBSINFILE INSERT INTO audit.CHECKING');

,ROWS_READ,ROWS_SKIPPED,ROWS_LOADED,ROWS_REJECTED,ROWS_DELETED,ROWS_COMMITTED,ROWS_PARTITIONED,NUM_AGENTINFO_ENTRIES,MSG_RETRIEVAL,MSG_REMOVAL
0,17,0,17,0,0,17,None,None,None,None


In [116]:
%sql CALL ADMIN_CMD('LOAD FROM /home/db2inst1/sqllib/security/auditdata/audit_del_files/context.del OF DEL MODIFIED BY DELPRIORITYCHAR  LOBSINFILE INSERT INTO audit.CONTEXT');


,ROWS_READ,ROWS_SKIPPED,ROWS_LOADED,ROWS_REJECTED,ROWS_DELETED,ROWS_COMMITTED,ROWS_PARTITIONED,NUM_AGENTINFO_ENTRIES,MSG_RETRIEVAL,MSG_REMOVAL
0,0,0,0,0,0,0,None,None,None,None


In [117]:
%sql CALL ADMIN_CMD('LOAD FROM /home/db2inst1/sqllib/security/auditdata/audit_del_files/execute.del OF DEL MODIFIED BY DELPRIORITYCHAR  LOBSINFILE INSERT INTO audit.EXECUTE');


,ROWS_READ,ROWS_SKIPPED,ROWS_LOADED,ROWS_REJECTED,ROWS_DELETED,ROWS_COMMITTED,ROWS_PARTITIONED,NUM_AGENTINFO_ENTRIES,MSG_RETRIEVAL,MSG_REMOVAL
0,2123,0,2123,0,0,2123,None,None,None,None


In [118]:
%sql CALL ADMIN_CMD('LOAD FROM /home/db2inst1/sqllib/security/auditdata/audit_del_files/objmaint.del OF DEL MODIFIED BY DELPRIORITYCHAR  LOBSINFILE INSERT INTO audit.OBJMAINT');


,ROWS_READ,ROWS_SKIPPED,ROWS_LOADED,ROWS_REJECTED,ROWS_DELETED,ROWS_COMMITTED,ROWS_PARTITIONED,NUM_AGENTINFO_ENTRIES,MSG_RETRIEVAL,MSG_REMOVAL
0,1,0,1,0,0,1,None,None,None,None


In [119]:
%sql CALL ADMIN_CMD('LOAD FROM /home/db2inst1/sqllib/security/auditdata/audit_del_files/secmaint.del OF DEL MODIFIED BY DELPRIORITYCHAR  LOBSINFILE INSERT INTO audit.SECMAINT');


,ROWS_READ,ROWS_SKIPPED,ROWS_LOADED,ROWS_REJECTED,ROWS_DELETED,ROWS_COMMITTED,ROWS_PARTITIONED,NUM_AGENTINFO_ENTRIES,MSG_RETRIEVAL,MSG_REMOVAL
0,0,0,0,0,0,0,None,None,None,None


In [120]:
%sql CALL ADMIN_CMD('LOAD FROM /home/db2inst1/sqllib/security/auditdata/audit_del_files/sysadmin.del OF DEL MODIFIED BY DELPRIORITYCHAR  LOBSINFILE INSERT INTO audit.SYSADMIN');


,ROWS_READ,ROWS_SKIPPED,ROWS_LOADED,ROWS_REJECTED,ROWS_DELETED,ROWS_COMMITTED,ROWS_PARTITIONED,NUM_AGENTINFO_ENTRIES,MSG_RETRIEVAL,MSG_REMOVAL
0,16,0,16,0,0,16,None,None,None,None


In [121]:
%sql CALL ADMIN_CMD('LOAD FROM /home/db2inst1/sqllib/security/auditdata/audit_del_files/validate.del OF DEL MODIFIED BY DELPRIORITYCHAR  LOBSINFILE INSERT INTO audit.VALIDATE');


,ROWS_READ,ROWS_SKIPPED,ROWS_LOADED,ROWS_REJECTED,ROWS_DELETED,ROWS_COMMITTED,ROWS_PARTITIONED,NUM_AGENTINFO_ENTRIES,MSG_RETRIEVAL,MSG_REMOVAL
0,0,0,0,0,0,0,None,None,None,None


In [122]:
%%sql -grid
SELECT * 
    FROM audit.execute
    WHERE event = 'STATEMENT'
    AND activitytype = 'WRITE_DML'

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [112]:
%%sql -grid
SELECT * 
    FROM audit.checking
    

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [81]:
%sql delete from audit.execute

Command completed.


In [72]:
%sql CONNECT RESET

Connection closed.


[Back to Top](#top)

## Effect of switching user <a id='user_switch'></a>

If a user is switched within a trusted connection, no remnants of the original user are left behind. In this case, the audit policies associated with the original user are no longer considered, and the applicable audit policies are re-evaluated according to the new user. Any audit policy associated with the trusted connection is still in effect.

If a SET SESSION USER statement is used, only the session authorization ID is switched. The audit policy of the authorization ID of the original user (the system authorization ID) remains in effect and the audit policy of the new user is used as well. If multiple SET SESSION USER statements are issued within a session, only the audit policies associated with the original user (the system authorization ID) and the current user (the session authorization ID) are considered.

In our HMO scenario we often used the `SET SESSION USER ` statement. Have a look into the audit tables and see the effects.

[Back to Top](#top)

## Clean all <a id='clean'></a>

In [30]:
%%sql
DROP TABLE AUDIT;
DROP TABLE CHECKING;
DROP TABLE OBJMAINT;
DROP TABLE SECMAINT;
DROP TABLE SYSADMIN;
DROP TABLE VALIDATE;
DROP TABLE CONTEXT;
DROP TABLE EXECUTE;

Command completed.


In [55]:
%%sql 
AUDIT SYSADM REMOVE POLICY;
AUDIT DBADM REMOVE POLICY;

Command completed.
